## CHAPTER 4: word2vec 속도 개선

In [1]:
import warnings
warnings.filterwarnings(action='ignore')

### 4.1 word2vec 개선 ①
+ **입력층의 원핫 표현**과 **가중치 행렬 W<sub>in</sub>**의 곱 계산

In [3]:
import numpy as np

class Embedding:
    def __init__(self, W):
        self.params = [W]
        self.grads = [np.zeros_like(W)]
        self.idx = None
    
    def forward(self, idx):
        W, = self.grads
        self.idx = idx
        out = W[idx]
        return out
    
    def backward(self, dout):
        dW, = self.grads
        dW[...] = 0
        
        for i, word_id in enumerate(self.idx):
            dW[word_id] += dout[i]
        
        return None

### 4.2 word2vec 개선 ②
+ **은닉층**과 **가중치 행렬 W<sub>out</sub>**의 곱
+ **Softmax 계층**의 계산

In [4]:
class EmbeddingDot:
    def __init__(self, W):
        self.embed = Embedding(W)
        self.params = self.embed.params
        self.grads = self.embed.grads
        self.cache = None
    
    def forward(self, h, idx):
        target_W = self.embed.forward(idx)
        out = np.sum(target_W * h, axis=1)
        
        self.cache = (h, target_W)
        return out
    
    def backward(self, dout):
        h, target_W = self.cache
        dout = dout.reshape(dout.shape[0], 1)
        
        dtarget_W = dout * h
        self.embed.backward(dtarget_W)
        dh = dout * target_W
        return dh

#### 4.2.5 네거티브 샘플링

긍정적 예를 타깃으로 한 경우의 손실을 구하고, 그와 동시에 부정적 예를 몇 개 샘플링하여 마찬가지로 손실을 구합니다.

그리고 각각의 데이터의 손실을 더한 값을 최종 손실로 합니다.

#### 4.2.7 네거티브 샘플링 구현

In [5]:
import sys
sys.path.append('exam_code')
from ch04.negative_sampling_layer import EmbeddingDot, UnigramSampler
from common.layers import SigmoidWithLoss

class NegativeSamplingLoss:
    def __init__(self, W, corpus, power=0.75, sample_size=5):
        self.sample_size = sample_size
        self.sampler = UnigramSampler(corpus, power, sample_size)
        self.loss_layers = [SigmoidWithLoss() for _ in range(sample_size+1)]
        self.embed_dot_layers = [EmbeddingDot(W) for _ in range(sample_size+1)]
        
        self.params, self.grads = [], []
        for layer in self.embed_dot_layers:
            self.params += layer.params
            self.grads  += layer.grads
        
    def forward(self, h, target):
        batch_size = target.shape[0]
        negative_sample = self.sampler.get_negative_sample(target)
        
        # 긍정적 예 순전파
        score = self.embed_dot_layers[0].forward(h, target)
        correct_label = np.ones(batch_size, dtype=np.int32)
        loss = self.loss_layers[0].forward(score, correct_label)
        
        # 부정적 예 순전파
        negative_label = np.zeros(batch_size, dtype=np.int32)
        for i in range(self.sample_size):
            negative_target = negative_sample[:, i]
            score = self.embed_dot_layers[i+1].forward(h, negative_target)
            loss += self.loss_layers[i+1].forward(score, negative_label)
        return loss
    
    def backward(self, dout=1):
        dh = 0
        for l0, l1 in zip(self.loss_layers, self.embed_dot_layers):
            dscore = l0.backward(dout)
            dh += l1.backward(dscore)
        return dh

### 4.3 개선판 word2vec 학습

#### 4.3.1 CBOW 모델 구현

In [6]:
import sys
sys.path.append('exam_code')
from common.layers import Embedding
from ch04.negative_sampling_layer import NegativeSamplingLoss

class CBOW:
    def __init__(self, vocab_size, hidden_size, window_size, corpus):
        V, H = vocab_size, hidden_size
        
        # 가중치 초기화
        W_in  = 0.01 * np.random.randn(V, H).astype('f')
        W_out = 0.01 * np.random.randn(V, H).astype('f')
        
        # 계층 생성
        self.in_layers = []
        for i in range(2 * window_size):
            layer = Embedding(W_in)
            self.in_layers.append(layer)
        self.ns_loss = NegativeSamplingLoss(W_out, corpus, power=0.75, sample_size=5)
        
        # 모든 가중치와 기울기를 배열에 모은다.
        layers = self.in_layers + [self.ns_loss]
        self.params, self.grads = [], []
        for layer in layers:
            self.params += layer.params
            self.grads  += layer.grads
        
        # 인스턴스 변수에 단어의 분산 표현을 저장한다.
        self.word_vecs = W_in
        
    def forward(self, contexts, target):
        h = 0
        for i, layer in enumerate(self.in_layers):
            h += layer.forward(contexts[:, i])
        h *= 1 / len(self.in_layers)
        loss = self.ns_loss.forward(h, target)
        return loss
        
    def backward(self, dout=1):
        dout = self.ns_loss.backward(dout)
        dout *= 1 / len(self.in_layers)
        for layer in self,in_layers:
            layer.backward(dout)
        return None

#### 4.3.2 CBOW 모델 학습 코드

In [ ]:
import sys
sys.path.append('exam_code')
import numpy as np
import pickle
from common.trainer import Trainer
from common.optimizer import Adam
from ch04.cbow import CBOW
from common.util import create_contexts_target
from dataset import ptb

# 하이퍼파라미터 설정
window_size = 5
hidden_size = 100
batch_size  = 100
max_epoch   = 10

# 데이터 읽기
corpus, word_to_id, id_to_word = ptb.load_data('train')
vocab_size = len(word_to_id)

contexts, target = create_contexts_target(corpus, window_size)

# 모델 등 생성
model     = CBOW(vocab_size, hidden_size, window_size, corpus)
optimizer = Adam()
trainer   = Trainer(model, optimizer)

# 학습 시작
trainer.fit(contexts, target, max_epoch, batch_size)
trainer.plot()

# 나중에 사용할 수 있도록 필요한 데이터 저장
word_vecs = model.word_vecs
params = {}
params['word_vecs']  = word_vecs.astype('np.float16')
params['word_to_id'] = word_to_id
params['id_to_word'] = id_to_word

pkl_file = 'cbow_params.pkl'
with open(pkl_file, 'wb') as f:
    pickle.dump(params, f, -1)

| 에폭 1 |  반복 1 / 9295 | 시간 0[s] | 손실 4.16
| 에폭 1 |  반복 21 / 9295 | 시간 1[s] | 손실 4.16
| 에폭 1 |  반복 41 / 9295 | 시간 3[s] | 손실 4.15
| 에폭 1 |  반복 61 / 9295 | 시간 4[s] | 손실 4.12
| 에폭 1 |  반복 81 / 9295 | 시간 6[s] | 손실 4.05
| 에폭 1 |  반복 101 / 9295 | 시간 7[s] | 손실 3.93
| 에폭 1 |  반복 121 / 9295 | 시간 9[s] | 손실 3.77
| 에폭 1 |  반복 141 / 9295 | 시간 10[s] | 손실 3.62
| 에폭 1 |  반복 161 / 9295 | 시간 12[s] | 손실 3.50
| 에폭 1 |  반복 181 / 9295 | 시간 13[s] | 손실 3.35
| 에폭 1 |  반복 201 / 9295 | 시간 15[s] | 손실 3.24
| 에폭 1 |  반복 221 / 9295 | 시간 16[s] | 손실 3.17
| 에폭 1 |  반복 241 / 9295 | 시간 18[s] | 손실 3.08
| 에폭 1 |  반복 261 / 9295 | 시간 19[s] | 손실 3.03
| 에폭 1 |  반복 281 / 9295 | 시간 21[s] | 손실 2.97
| 에폭 1 |  반복 301 / 9295 | 시간 22[s] | 손실 2.90
| 에폭 1 |  반복 321 / 9295 | 시간 24[s] | 손실 2.87
| 에폭 1 |  반복 341 / 9295 | 시간 25[s] | 손실 2.83
| 에폭 1 |  반복 361 / 9295 | 시간 27[s] | 손실 2.82
| 에폭 1 |  반복 381 / 9295 | 시간 28[s] | 손실 2.78
| 에폭 1 |  반복 401 / 9295 | 시간 30[s] | 손실 2.75
| 에폭 1 |  반복 421 / 9295 | 시간 31[s] | 손실 2.74
| 에폭 1 |  반복 441 / 9295

| 에폭 1 |  반복 3561 / 9295 | 시간 258[s] | 손실 2.42
| 에폭 1 |  반복 3581 / 9295 | 시간 260[s] | 손실 2.43
| 에폭 1 |  반복 3601 / 9295 | 시간 261[s] | 손실 2.41
| 에폭 1 |  반복 3621 / 9295 | 시간 262[s] | 손실 2.41
| 에폭 1 |  반복 3641 / 9295 | 시간 264[s] | 손실 2.41
| 에폭 1 |  반복 3661 / 9295 | 시간 265[s] | 손실 2.38
| 에폭 1 |  반복 3681 / 9295 | 시간 267[s] | 손실 2.43
| 에폭 1 |  반복 3701 / 9295 | 시간 268[s] | 손실 2.43
| 에폭 1 |  반복 3721 / 9295 | 시간 270[s] | 손실 2.42
| 에폭 1 |  반복 3741 / 9295 | 시간 271[s] | 손실 2.43
| 에폭 1 |  반복 3761 / 9295 | 시간 272[s] | 손실 2.38
| 에폭 1 |  반복 3781 / 9295 | 시간 274[s] | 손실 2.44
| 에폭 1 |  반복 3801 / 9295 | 시간 275[s] | 손실 2.40
| 에폭 1 |  반복 3821 / 9295 | 시간 277[s] | 손실 2.39
| 에폭 1 |  반복 3841 / 9295 | 시간 278[s] | 손실 2.42
| 에폭 1 |  반복 3861 / 9295 | 시간 280[s] | 손실 2.43
| 에폭 1 |  반복 3881 / 9295 | 시간 281[s] | 손실 2.41
| 에폭 1 |  반복 3901 / 9295 | 시간 282[s] | 손실 2.39
| 에폭 1 |  반복 3921 / 9295 | 시간 284[s] | 손실 2.39
| 에폭 1 |  반복 3941 / 9295 | 시간 285[s] | 손실 2.39
| 에폭 1 |  반복 3961 / 9295 | 시간 287[s] | 손실 2.38
| 에폭 1 |  반복 

| 에폭 1 |  반복 7061 / 9295 | 시간 512[s] | 손실 2.25
| 에폭 1 |  반복 7081 / 9295 | 시간 514[s] | 손실 2.29
| 에폭 1 |  반복 7101 / 9295 | 시간 515[s] | 손실 2.27
| 에폭 1 |  반복 7121 / 9295 | 시간 517[s] | 손실 2.25
| 에폭 1 |  반복 7141 / 9295 | 시간 518[s] | 손실 2.27
| 에폭 1 |  반복 7161 / 9295 | 시간 519[s] | 손실 2.26
| 에폭 1 |  반복 7181 / 9295 | 시간 521[s] | 손실 2.28
| 에폭 1 |  반복 7201 / 9295 | 시간 522[s] | 손실 2.28
| 에폭 1 |  반복 7221 / 9295 | 시간 524[s] | 손실 2.29
| 에폭 1 |  반복 7241 / 9295 | 시간 525[s] | 손실 2.28
| 에폭 1 |  반복 7261 / 9295 | 시간 527[s] | 손실 2.25
| 에폭 1 |  반복 7281 / 9295 | 시간 528[s] | 손실 2.26
| 에폭 1 |  반복 7301 / 9295 | 시간 529[s] | 손실 2.27
| 에폭 1 |  반복 7321 / 9295 | 시간 531[s] | 손실 2.27
| 에폭 1 |  반복 7341 / 9295 | 시간 532[s] | 손실 2.27
| 에폭 1 |  반복 7361 / 9295 | 시간 534[s] | 손실 2.29
| 에폭 1 |  반복 7381 / 9295 | 시간 535[s] | 손실 2.30
| 에폭 1 |  반복 7401 / 9295 | 시간 537[s] | 손실 2.29
| 에폭 1 |  반복 7421 / 9295 | 시간 538[s] | 손실 2.26
| 에폭 1 |  반복 7441 / 9295 | 시간 539[s] | 손실 2.27
| 에폭 1 |  반복 7461 / 9295 | 시간 541[s] | 손실 2.28
| 에폭 1 |  반복 

| 에폭 2 |  반복 1281 / 9295 | 시간 767[s] | 손실 2.18
| 에폭 2 |  반복 1301 / 9295 | 시간 768[s] | 손실 2.17
| 에폭 2 |  반복 1321 / 9295 | 시간 770[s] | 손실 2.19
| 에폭 2 |  반복 1341 / 9295 | 시간 771[s] | 손실 2.17
| 에폭 2 |  반복 1361 / 9295 | 시간 773[s] | 손실 2.18
| 에폭 2 |  반복 1381 / 9295 | 시간 774[s] | 손실 2.17
| 에폭 2 |  반복 1401 / 9295 | 시간 776[s] | 손실 2.19
| 에폭 2 |  반복 1421 / 9295 | 시간 777[s] | 손실 2.18
| 에폭 2 |  반복 1441 / 9295 | 시간 779[s] | 손실 2.16
| 에폭 2 |  반복 1461 / 9295 | 시간 780[s] | 손실 2.16
| 에폭 2 |  반복 1481 / 9295 | 시간 782[s] | 손실 2.18
| 에폭 2 |  반복 1501 / 9295 | 시간 783[s] | 손실 2.13
| 에폭 2 |  반복 1521 / 9295 | 시간 785[s] | 손실 2.14
| 에폭 2 |  반복 1541 / 9295 | 시간 786[s] | 손실 2.17
| 에폭 2 |  반복 1561 / 9295 | 시간 787[s] | 손실 2.13
| 에폭 2 |  반복 1581 / 9295 | 시간 789[s] | 손실 2.17
| 에폭 2 |  반복 1601 / 9295 | 시간 790[s] | 손실 2.12
| 에폭 2 |  반복 1621 / 9295 | 시간 792[s] | 손실 2.13
| 에폭 2 |  반복 1641 / 9295 | 시간 793[s] | 손실 2.13
| 에폭 2 |  반복 1661 / 9295 | 시간 795[s] | 손실 2.16
| 에폭 2 |  반복 1681 / 9295 | 시간 796[s] | 손실 2.16
| 에폭 2 |  반복 

| 에폭 2 |  반복 4761 / 9295 | 시간 1022[s] | 손실 2.13
| 에폭 2 |  반복 4781 / 9295 | 시간 1023[s] | 손실 2.07
| 에폭 2 |  반복 4801 / 9295 | 시간 1025[s] | 손실 2.12
| 에폭 2 |  반복 4821 / 9295 | 시간 1026[s] | 손실 2.08
| 에폭 2 |  반복 4841 / 9295 | 시간 1027[s] | 손실 2.09
| 에폭 2 |  반복 4861 / 9295 | 시간 1029[s] | 손실 2.09
| 에폭 2 |  반복 4881 / 9295 | 시간 1030[s] | 손실 2.06
| 에폭 2 |  반복 4901 / 9295 | 시간 1032[s] | 손실 2.10
| 에폭 2 |  반복 4921 / 9295 | 시간 1033[s] | 손실 2.11
| 에폭 2 |  반복 4941 / 9295 | 시간 1035[s] | 손실 2.05
| 에폭 2 |  반복 4961 / 9295 | 시간 1036[s] | 손실 2.08
| 에폭 2 |  반복 4981 / 9295 | 시간 1038[s] | 손실 2.09
| 에폭 2 |  반복 5001 / 9295 | 시간 1039[s] | 손실 2.09
| 에폭 2 |  반복 5021 / 9295 | 시간 1041[s] | 손실 2.09
| 에폭 2 |  반복 5041 / 9295 | 시간 1042[s] | 손실 2.07
| 에폭 2 |  반복 5061 / 9295 | 시간 1044[s] | 손실 2.05
| 에폭 2 |  반복 5081 / 9295 | 시간 1045[s] | 손실 2.10
| 에폭 2 |  반복 5101 / 9295 | 시간 1046[s] | 손실 2.09
| 에폭 2 |  반복 5121 / 9295 | 시간 1048[s] | 손실 2.05
| 에폭 2 |  반복 5141 / 9295 | 시간 1049[s] | 손실 2.08
| 에폭 2 |  반복 5161 / 9295 | 시간 1051[s] | 

| 에폭 2 |  반복 8181 / 9295 | 시간 1272[s] | 손실 2.05
| 에폭 2 |  반복 8201 / 9295 | 시간 1273[s] | 손실 2.04
| 에폭 2 |  반복 8221 / 9295 | 시간 1274[s] | 손실 2.04
| 에폭 2 |  반복 8241 / 9295 | 시간 1276[s] | 손실 1.99
| 에폭 2 |  반복 8261 / 9295 | 시간 1277[s] | 손실 1.99
| 에폭 2 |  반복 8281 / 9295 | 시간 1279[s] | 손실 2.01
| 에폭 2 |  반복 8301 / 9295 | 시간 1280[s] | 손실 2.05
| 에폭 2 |  반복 8321 / 9295 | 시간 1282[s] | 손실 2.03
| 에폭 2 |  반복 8341 / 9295 | 시간 1283[s] | 손실 2.02
| 에폭 2 |  반복 8361 / 9295 | 시간 1285[s] | 손실 2.05
| 에폭 2 |  반복 8381 / 9295 | 시간 1286[s] | 손실 2.03
| 에폭 2 |  반복 8401 / 9295 | 시간 1287[s] | 손실 2.06
| 에폭 2 |  반복 8421 / 9295 | 시간 1289[s] | 손실 2.03
| 에폭 2 |  반복 8441 / 9295 | 시간 1290[s] | 손실 2.05
| 에폭 2 |  반복 8461 / 9295 | 시간 1292[s] | 손실 2.01
| 에폭 2 |  반복 8481 / 9295 | 시간 1293[s] | 손실 2.04
| 에폭 2 |  반복 8501 / 9295 | 시간 1295[s] | 손실 2.02
| 에폭 2 |  반복 8521 / 9295 | 시간 1296[s] | 손실 2.04
| 에폭 2 |  반복 8541 / 9295 | 시간 1298[s] | 손실 2.00
| 에폭 2 |  반복 8561 / 9295 | 시간 1299[s] | 손실 2.04
| 에폭 2 |  반복 8581 / 9295 | 시간 1300[s] | 

| 에폭 3 |  반복 2321 / 9295 | 시간 1519[s] | 손실 1.98
| 에폭 3 |  반복 2341 / 9295 | 시간 1520[s] | 손실 1.95
| 에폭 3 |  반복 2361 / 9295 | 시간 1522[s] | 손실 1.97
| 에폭 3 |  반복 2381 / 9295 | 시간 1523[s] | 손실 1.94
| 에폭 3 |  반복 2401 / 9295 | 시간 1525[s] | 손실 1.88
| 에폭 3 |  반복 2421 / 9295 | 시간 1526[s] | 손실 1.91
| 에폭 3 |  반복 2441 / 9295 | 시간 1528[s] | 손실 1.93
| 에폭 3 |  반복 2461 / 9295 | 시간 1529[s] | 손실 1.99
| 에폭 3 |  반복 2481 / 9295 | 시간 1530[s] | 손실 1.95
| 에폭 3 |  반복 2501 / 9295 | 시간 1532[s] | 손실 1.96
| 에폭 3 |  반복 2521 / 9295 | 시간 1533[s] | 손실 1.91
| 에폭 3 |  반복 2541 / 9295 | 시간 1535[s] | 손실 1.93
| 에폭 3 |  반복 2561 / 9295 | 시간 1536[s] | 손실 1.98
| 에폭 3 |  반복 2581 / 9295 | 시간 1538[s] | 손실 1.97
| 에폭 3 |  반복 2601 / 9295 | 시간 1539[s] | 손실 1.94
| 에폭 3 |  반복 2621 / 9295 | 시간 1540[s] | 손실 1.96
| 에폭 3 |  반복 2641 / 9295 | 시간 1542[s] | 손실 1.93
| 에폭 3 |  반복 2661 / 9295 | 시간 1543[s] | 손실 1.93
| 에폭 3 |  반복 2681 / 9295 | 시간 1545[s] | 손실 1.93
| 에폭 3 |  반복 2701 / 9295 | 시간 1546[s] | 손실 1.88
| 에폭 3 |  반복 2721 / 9295 | 시간 1548[s] | 

| 에폭 3 |  반복 5741 / 9295 | 시간 1766[s] | 손실 1.92
| 에폭 3 |  반복 5761 / 9295 | 시간 1768[s] | 손실 1.93
| 에폭 3 |  반복 5781 / 9295 | 시간 1769[s] | 손실 1.91
| 에폭 3 |  반복 5801 / 9295 | 시간 1770[s] | 손실 1.92
| 에폭 3 |  반복 5821 / 9295 | 시간 1772[s] | 손실 1.90
| 에폭 3 |  반복 5841 / 9295 | 시간 1773[s] | 손실 1.92
| 에폭 3 |  반복 5861 / 9295 | 시간 1775[s] | 손실 1.93
| 에폭 3 |  반복 5881 / 9295 | 시간 1776[s] | 손실 1.93
| 에폭 3 |  반복 5901 / 9295 | 시간 1778[s] | 손실 1.92
| 에폭 3 |  반복 5921 / 9295 | 시간 1779[s] | 손실 1.94
| 에폭 3 |  반복 5941 / 9295 | 시간 1781[s] | 손실 1.94
| 에폭 3 |  반복 5961 / 9295 | 시간 1782[s] | 손실 1.92
| 에폭 3 |  반복 5981 / 9295 | 시간 1783[s] | 손실 1.87
| 에폭 3 |  반복 6001 / 9295 | 시간 1785[s] | 손실 1.91
| 에폭 3 |  반복 6021 / 9295 | 시간 1786[s] | 손실 1.93
| 에폭 3 |  반복 6041 / 9295 | 시간 1788[s] | 손실 1.94
| 에폭 3 |  반복 6061 / 9295 | 시간 1789[s] | 손실 1.92
| 에폭 3 |  반복 6081 / 9295 | 시간 1791[s] | 손실 1.94
| 에폭 3 |  반복 6101 / 9295 | 시간 1792[s] | 손실 1.91
| 에폭 3 |  반복 6121 / 9295 | 시간 1793[s] | 손실 1.90
| 에폭 3 |  반복 6141 / 9295 | 시간 1795[s] | 

| 에폭 3 |  반복 9161 / 9295 | 시간 2020[s] | 손실 1.91
| 에폭 3 |  반복 9181 / 9295 | 시간 2021[s] | 손실 1.91
| 에폭 3 |  반복 9201 / 9295 | 시간 2023[s] | 손실 1.90
| 에폭 3 |  반복 9221 / 9295 | 시간 2025[s] | 손실 1.89
| 에폭 3 |  반복 9241 / 9295 | 시간 2026[s] | 손실 1.88
| 에폭 3 |  반복 9261 / 9295 | 시간 2028[s] | 손실 1.92
| 에폭 3 |  반복 9281 / 9295 | 시간 2030[s] | 손실 1.83
| 에폭 4 |  반복 1 / 9295 | 시간 2031[s] | 손실 1.89
| 에폭 4 |  반복 21 / 9295 | 시간 2033[s] | 손실 1.82
| 에폭 4 |  반복 41 / 9295 | 시간 2034[s] | 손실 1.79
| 에폭 4 |  반복 61 / 9295 | 시간 2036[s] | 손실 1.85
| 에폭 4 |  반복 81 / 9295 | 시간 2038[s] | 손실 1.80
| 에폭 4 |  반복 101 / 9295 | 시간 2039[s] | 손실 1.83
| 에폭 4 |  반복 121 / 9295 | 시간 2041[s] | 손실 1.79
| 에폭 4 |  반복 141 / 9295 | 시간 2043[s] | 손실 1.82
| 에폭 4 |  반복 161 / 9295 | 시간 2044[s] | 손실 1.86
| 에폭 4 |  반복 181 / 9295 | 시간 2046[s] | 손실 1.84
| 에폭 4 |  반복 201 / 9295 | 시간 2048[s] | 손실 1.83
| 에폭 4 |  반복 221 / 9295 | 시간 2049[s] | 손실 1.84
| 에폭 4 |  반복 241 / 9295 | 시간 2051[s] | 손실 1.82
| 에폭 4 |  반복 261 / 9295 | 시간 2053[s] | 손실 1.82
| 에폭 4 |  반복

#### 4.3.3 CBOW 모델 평가

In [ ]:
import sys
sys.path.append('exam_code')
from common.util import most_similar
import pickle

pkl_file = 'cbow_params.pkl'

with open(pkl_file, 'rb') as f:
    params = pickle.load(f)
    word_vecs  = params['word_vecs']
    word_to_id = params['word_to_id']
    id_to_word = params['id_to_word']
    
querys = ['you', 'year', 'car', 'toyota']
for query in querys:
    most_similar(query, word_to_id, id_to_word, word_vecs, top=5)

In [ ]:
import sys
sys.path.append('exam_code')
from common.util import analogy

analogy('king', 'man', 'queen', word_to_id, id_to_word, word_vecs)
analogy('take', 'took', 'go', word_to_id, id_to_word, word_vecs)
analogy('car', 'cars', 'child', word_to_id, id_to_word, word_vecs)
analogy('good', 'better', 'bad', word_to_id, id_to_word, word_vecs)